###### imports

In [1]:
import os
import pandas as pd
import numpy as np

from pathlib import Path

import scipy

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

import synthetic_data

2023-09-01 17:03:03.492251: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-01 17:03:03.493721: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 17:03:03.525074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 17:03:03.526285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 17:03:04.057173: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Check Path

In [2]:
DATA_PATH_RISING_WEDGE = Path("data/patterns/rising_wedge")
DATA_PATH_RISING_WEDGE.exists()

data = {filepath.stem: pd.read_csv(filepath) for filepath in DATA_PATH_RISING_WEDGE.glob("*.csv")}

## Create Tensor and Label lists from real data

In [3]:
X_real = []
y_real = []

for key, df in data.items():
    X_real.append(df[['Open', 'High', 'Low', 'Close']].values)
    y_real.append(df.loc[0, ['Start Date', 'End Date', 'Pattern']].values)

## Create Synthetic Data with pattern

In [4]:
amount = int(len(X_real) * 0.40)
amount_25 = int(len(y_real) * 0.25)

X_synthetic, y_synthetic = synthetic_data.gen_x_y(l=amount, pattern="rising_wedge", noise=True, general=False)

## Take Real data with no pattern

In [5]:
DATA_PATH_DOWNTREND = Path("data/patterns/downtrend")
DATA_PATH_DOWNTREND.exists()

downtrend_data = {filepath.stem: pd.read_csv(filepath) for filepath in DATA_PATH_DOWNTREND.glob("*.csv")}

X_no_pattern = []
y_no_pattern = []

for key, df in downtrend_data.items():
    X_no_pattern.append(df[['Open', 'High', 'Low', 'Close']].values)
    y_no_pattern.append(df.loc[0, ['Start Date', 'End Date', 'Pattern']].values)

X_no_pattern = X_no_pattern[:amount_25]
y_no_pattern = y_no_pattern[:amount_25]

In [6]:
total = find_max_length(X_real, X_synthetic, X_no_pattern)
total

NameError: name 'find_max_length' is not defined

## Join lists

In [7]:
X_all = X_real + X_synthetic + X_no_pattern
y_all = y_real + y_synthetic + y_no_pattern

## Padding

In [8]:
def find_longest_array(array):
    largest = 0
    for a in array:
        if len(a) > largest:
            largest = len(a)
    return largest

test = find_longest_array(X_all)

In [9]:
X_pad = pad_sequences(X_all, dtype='float32', padding='post', value=-100)

### Test padding

In [11]:
X_pad.shape

(1139, 502, 4)

## Train Test Split

In [12]:
y_all = np.array(y_all)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(797, 502, 4)

(342, 502, 4)

(797, 3)

(342, 3)